In [ ]:
import huggingface_hub
hf_token = '' # put your User Access Tokens here
# ابتدا login کنید
huggingface_hub.login(token=hf_token)

# سپس وضعیت ورود را بررسی کنید
!hf auth whoami

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


user:  AM-Nateghi


In [11]:
from transformers import AutoTokenizer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-pt")  # یا gemma-7b بسته به مدل
total_tokens = 0

# ========== بارگذاری دیتاست ==========
dataset = load_dataset(
    "json", data_files={"train": "assets/qa_train.json", "test": "assets/qa_test.json"}
)

for sample in dataset['train']:  # فرض بر اینه که dataset لیستی از دیکشنری‌های {"question": ..., "answer": ...} هست
    text = sample["input"] + " " + sample["output"]
    tokens = tokenizer.encode(text, add_special_tokens=True)
    total_tokens += len(tokens)

print("Total token count:", total_tokens)

Total token count: 898607


In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

torch.cuda.empty_cache()

# بارگذاری tokenizer
tokenizer = AutoTokenizer.from_pretrained("saved_models/gemma_final_model")
# بارگذاری مدل پایه
base_model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-pt")
# بارگذاری LoRA روی مدل پایه
model = PeftModel.from_pretrained(base_model, "saved_models/gemma_final_model")
# انتقال به GPU
model.to("cuda")


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForCausalLM(
      (model): Gemma3TextModel(
        (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma3DecoderLayer(
            (self_attn): Gemma3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1152, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1152, out_features=256, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=256, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
      

In [22]:
output = tokenizer.decode(
    model.generate(
        **tokenizer("پرسش: حتی اگه بپذیریم که اصل قانون ازدواج موقّت پسندیدهست؛ ولی چرا در اسلام، مرد در ازدواج موقت هیچ محدودیتی نداره و میتونه با بینهایت زن ازدواج موقت کنه؟ \nپاسخ: ", return_tensors="pt").to(
            "cuda"
        ),
        max_new_tokens=512,
    )[0],
    skip_special_tokens=True
)
print(output)

پرسش: حتی اگه بپذیریم که اصل قانون ازدواج موقّت پسندیدهست؛ ولی چرا در اسلام، مرد در ازدواج موقت هیچ محدودیتی نداره و میتونه با بینهایت زن ازدواج موقت کنه؟ 
پاسخ: 

برای پاسخ به این سوال باید به این نکته اشاره کنیم که هدف از قانون ازدواج موقت، محدود کردن آسیبهای اجتماعی که میتونه از این ازدواجها نشون داده بشه و از این طریق ریشهگیری و درمان اونها در جامعه رو تسهیل کنه. 

برای رسیدن به چنین اهدافی باید ازدواج و به هم ریختگی آن رو در جامعه کنار گذاشت و فقط رو این امر تأکید کرد که ازدواج رو برای مسائل خاصی که در زندگی مشترک مورد نیاز در نظر میگیریم، برگزار کنیم.
